In [ ]:
import pandas as pd
import json

import boto3
from eliot import log_message
from sqlalchemy import create_engine
from sqlalchemy.engine.url import URL

In [ ]:
CLIENT = 'marquis'
START_DATE = '2021-05-01'
END_DATE = '2021-06-30'

In [ ]:
class DbEngine(object):
    """
    Fetch the credentials from AWS Secrets Manager.
    :return: DB connection to the respective database
    """

    def __init__(self, region_name='us-east-1'):
        self.session = boto3.session.Session()
        self.secrets_client = self.session.client(
            service_name='secretsmanager',
            region_name=region_name
        )

    def get_secrets(self, secret_name):
        """
        :return: Based on the environment get secrets for
        Client SQL db & Postgres Saivadb
        """
        log_message(message_type='info', action_type='get_secrets', secret_name=secret_name)
        db_info = json.loads(
            self.secrets_client.get_secret_value(SecretId=secret_name)[
                'SecretString'
            ]
        )
        return db_info

    def get_postgresdb_engine(self):
        """
        Based on the environment connects to the respective database
        :param client: client name
        :return: Saivadb Postgres engine
        """
        log_message(message_type='info', action_type='connect_to_postgresdb', client='SaivaDB')
        # Fetch credentials from AWS Secrets Manager
        postgresdb_info = self.get_secrets(secret_name=f'prod-saivadb')
        # Create DB URL
        saivadb_url = URL(
            drivername='postgresql',
            username=postgresdb_info['username'],
            password=postgresdb_info['password'],
            host=postgresdb_info['host'],
            port=postgresdb_info['port'],
            database=postgresdb_info['dbname'],
        )
        # Return Postgres Engine
        return create_engine(saivadb_url, echo=False)
    
    def get_sqldb_engine(self, clientdb_name):
        """
        Based on the environment connects to the respective database.
        Avante db is in client VPN hence we use different credentials.
        :param client: client name
        :return: Client SQL engine
        """
        log_message(message_type='info', action_type='connect_to_sqldb', client=clientdb_name)
        # Fetch credentials from AWS Secrets Manager
        if clientdb_name == 'avante':
            sqldb_info = self.get_secrets(secret_name=f'avantedb')
        else:
            sqldb_info = self.get_secrets(secret_name=f'prod-sqlserver')
            sqldb_info['dbname'] = clientdb_name

        # Create DB URL
        client_sqldb_url = URL(
            drivername='mssql+pyodbc',
            username=sqldb_info['username'],
            password=sqldb_info['password'],
            host=sqldb_info['host'],
            port=sqldb_info['port'],
            database=sqldb_info['dbname'],
            query={'driver': 'ODBC Driver 17 for SQL Server'},
        )
        # Return Sql Engine
        return create_engine(client_sqldb_url, echo=False)
    
    def verify_connectivity(self, engine):
        assert engine.execute('select 1').fetchall() is not None  # verify connectivity

engine = DbEngine()
client_engine =  engine.get_postgresdb_engine()


In [ ]:
query = f"""
with rh as (
    select ht.*,
    fa.facilityname,
    fp.masterpatientid,
    dp.modelid,
    dp.predictionrank,
    dp.show_in_report,
    fp.patientmrn,
    fp.firstname,
    fp.lastname
    from public.hospital_transfers ht
        left join public.facility_patient fp
        on ht.client = fp.client
        and ht.facilityid = fp.facilityid
        and ht.patientid = fp.patientid
            left join daily_predictions dp
            on ht.client = dp.client
            and ht.facilityid = dp.facilityid
            and (date(ht.dateoftransfer) - date(dp.censusdate)) <= 3
            and (date(ht.dateoftransfer) - date(dp.censusdate)) != 0
            and date(dp.censusdate) <= date(ht.dateoftransfer)
            and fp.masterpatientid = dp.masterpatientid
            left join facility fa
            on fa.facilityid = ht.facilityid
            and fa.client = ht.client
    where (dp.published = True or dp.published is null)
      and ht.dateoftransfer >= '2020-01-01 00:00:00'
      and (dp.experiment_group = True or dp.experiment_group is null)
      and fa.is_active=true
      and ht.client='{CLIENT}'
      and dp.censusdate between '{START_DATE}' and '{END_DATE}'
)
    SELECT rh.client,
           rh.facilityid,
           rh.facilityname,
           rh.modelid,
           rh.patientid,
           rh.masterpatientid,
           rh.patientmrn,
           rh.lastname,
           rh.firstname,
           rh.dateoftransfer,
           rh.planned,
           rh.transferreason,
           rh.otherreasonfortransfer,
           rh.outcome,
           rh.transferredto,
           rh.lengthofstay,
           rh.payertype,
           rh.payerdescription,
           min(predictionrank) as rank_cutoff,
           bool_or(rh.show_in_report) as show_in_report,
           (CASE
                WHEN bool_or(rh.show_in_report) IS NULL
                    THEN 0
                ELSE count(*)
               END
               ) as num_predictions
    FROM rh
    GROUP BY rh.client, rh.facilityid, rh.facilityname, rh.modelid,
             rh.patientid, rh.masterpatientid, rh.patientmrn, rh.lastname, rh.firstname, rh.dateoftransfer,
             rh.planned, rh.transferreason, rh.otherreasonfortransfer, rh.outcome,
             rh.transferredto, rh.lengthofstay, rh.payertype, rh.payerdescription
"""



In [ ]:
# creating a dataframe from the sql query
df = pd.read_sql(query, con = client_engine)
df = df[['client','facilityid','dateoftransfer','masterpatientid','rank_cutoff']]
df = df[(df['dateoftransfer']>='2021-05-01')&(df['dateoftransfer']<='2021-06-30')]
# dataframe contains all the transferred residents.
df['resident_transferred'] = 1

# sorting the dataframe w.r.t dateoftransfer, facilityid and rank_cutoff
df = df.sort_values(by=['dateoftransfer','facilityid','rank_cutoff'])
df.head()

In [ ]:
"""
following code snippet perform the below operation for each date and each facilityid - 
1) It merges(right join) the dataframe slice(each date, each facilityid) 
with another dataframe 'one_to_fifty' which contains single column of 1 to 50.
2) 'client','facilityid','dateoftransfer' is filled by ffill.
3) 'rank' is copied on 'rank_cutoff' and all the null values are filled by 0.
All the dataframe slices are appended together to form a single dataframe.
"""
combined_list =[]
one_to_fifty = pd.Series(range(1, 300+1))
one_to_fifty = pd.DataFrame({'rank':one_to_fifty})
facilityids = df.facilityid.unique().tolist()
for date in pd.date_range(start='2021-05-01', end='2021-06-30'):
    for facility in facilityids:
        specific_date_facility_df = df[(df['dateoftransfer']==date) & (df['facilityid']==facility)]
        
        if len(specific_date_facility_df):
            specific_date_facility_df = specific_date_facility_df.merge(one_to_fifty,left_on='rank_cutoff', right_on='rank', how='right')
            specific_date_facility_df.loc[:,['client','facilityid','dateoftransfer']] = specific_date_facility_df.loc[:,['client','facilityid','dateoftransfer']].ffill()
            specific_date_facility_df['resident_transferred'].fillna(0,inplace=True)
            specific_date_facility_df['rank_cutoff'] = specific_date_facility_df['rank']
            specific_date_facility_df = specific_date_facility_df.drop('rank',axis=1)
            specific_date_facility_df = specific_date_facility_df.sort_values(by=['dateoftransfer','facilityid','rank_cutoff'])
            combined_list.append(specific_date_facility_df)

In [ ]:
combined_df = pd.concat(combined_list)
combined_df = combined_df.sort_values(by=['dateoftransfer','facilityid','rank_cutoff'])
combined_df = combined_df.reset_index(drop=True)


In [ ]:
"""
'hospitalized_cumsum' signifies the cumulative summation of the patients transferred per facility per date
'total_relevant' signifies the total residents transferred to the hospital per facility per date
"""
def precision_recall_at_k(group):
    group.loc[:, "hospitalized_cumsum"] = group.resident_transferred.cumsum()
    group.loc[:, "total_relevant"] = group.resident_transferred.sum()
    return group.reset_index(drop=True)

performance_base = (
            combined_df.groupby(["client", "facilityid", "dateoftransfer"])
            .apply(precision_recall_at_k)
            .reset_index(drop=True)
        )


In [ ]:
# summation of the 'hospitalized_cumsum' grouped by client, facilityid and rank_cutoff.
predicted_df = performance_base.groupby(['client','facilityid','rank_cutoff']).hospitalized_cumsum.sum().reset_index(name='total_residents_predicted')
predicted_df.head()

In [ ]:
# summation of the 'total_relevant' grouped by client, facilityid and rank_cutoff.
transferred_df = performance_base.groupby(['client','facilityid','rank_cutoff']).total_relevant.sum().reset_index(name='total_residents_transferred')
transferred_df.head()

In [ ]:
# combining the above to dataframes and dividing total_residents_predicted by total_residents_transferred to get the recall.
recall_df = predicted_df.merge(transferred_df,how='inner',on=['client','facilityid','rank_cutoff'])
recall_df['recall_at_k'] = recall_df['total_residents_predicted']/recall_df['total_residents_transferred']
recall_df = recall_df[recall_df['rank_cutoff']<=50]
recall_df.head()

In [ ]:
sqlengine = engine.get_sqldb_engine('marquis')

census_query = f"""
    select facilityid, count(*)/30 as census_count
    from view_ods_daily_census_v2
    where censusdate>= '2021-06-01' and censusdate<= '2021-06-30'
    group by facilityid
"""

census_df = pd.read_sql(census_query, con = sqlengine)
census_df.head()

In [ ]:
recall_census_df = recall_df.merge(census_df,how='inner',on='facilityid')
recall_census_df['fac_id__census__res_transferred'] =  recall_census_df['facilityid'].astype(int).astype(str)+'__'+\
recall_census_df['census_count'].astype(str)+'__'+recall_census_df['total_residents_transferred'].astype(str)


In [ ]:
recall_census_df[(recall_census_df['facilityid']==33) & (recall_census_df['rank_cutoff']==45)]

In [ ]:
recall_census_df[(recall_census_df['facilityid']==1) & (recall_census_df['rank_cutoff']==15)]

In [ ]:
recall_census_df[(recall_census_df['facilityid']==19) & (recall_census_df['rank_cutoff']==30)]

In [ ]:
recall_census_df[(recall_census_df['facilityid']==38) & (recall_census_df['rank_cutoff']==15)]

In [ ]:
recall_census_df.to_csv('recall_at_every_rank.csv',index=False)